<a href="https://colab.research.google.com/github/afnanmumu/CV-parsing-using-spacy/blob/main/Using_GPU_CV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!nvidia-smi

Mon Jun 19 06:08:51 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install spacy_transformers
!pip install -U spacy
!pip install tensorrt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.8/190.8 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 63.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 75.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 84.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 41.9 MB/s eta 0:00:00
  Attempting uninstall: spacy
    Found existing installation: spacy 3.5.2
    Uninstalling spacy-3.5.2:
      Successfully uninstalled spacy-3.5.2
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple

In [ ]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json

In [ ]:
spacy.__version__

'3.5.3'

In [ ]:
cv_load = json.load(open('/content/drive/MyDrive/Red_Dot/spacy_CV/data/training/dataset.json','r'))

In [ ]:
len(cv_load)

1014

In [ ]:
import locale
print(locale.getpreferredencoding())

UTF-8


In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
!python -m spacy init fill-config /content/drive/MyDrive/Red_Dot/spacy_CV/data/training/base_config.cfg /content/drive/MyDrive/Red_Dot/spacy_CV/data/training/config.cfg

2023-06-19 06:11:21.364405: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Auto-filled config with all values
✔ Saved config
/content/drive/MyDrive/Red_Dot/spacy_CV/data/training/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
cv_load[0]

['\xa0 \xa0\nContact\nwww.linkedin.com/in/omjagri\n(LinkedIn)\nTop Skills\nphp\nMySQL\nJavaScript\nCertifications\nPhp & Js Om Prakash Jagri\nFull Stack Developer | PHP | Laravel | Vue Js\nKathmandu, Bāgmatī, Nepal\nSummary\nExperienced Developer with a demonstrated history of working in\nthe information technology and services industry. Skilled in Laravel,\nPHP, Cascading Style Sheets (CSS), JavaScript, vue js and MySQL.\nStrong engineering professional with a B.sc.CSIT(Bachelors of\nScience in Computer Science and Information Technology) focused\nin Computer Science from Tribhuvan University, Institute of Science\n& Tchnology. \nExperience\nSearchable Design LLC\nSoftware Developer\nJune 2021\xa0-\xa0Present\xa0 (1 year 7 months)\nNepal\nFull Stack Developer Laravel with Vue Js\nBenekiva\nTechnical Documentation\nSeptember 2021\xa0-\xa0Present\xa0 (1 year 4 months)\nUnited States\nBidhee\n3 years 9 months\nLaravel Developer\nMarch 2018\xa0-\xa0May 2021\xa0 (3 years 3 months)\nBaneswa

In [ ]:
def get_spacy_doc(file,data):
  nlp=spacy.blank('en')
  db=DocBin()

  for text,annot in tqdm(data):
    doc=nlp.make_doc(text)
    annot = annot['entities']
    ents = []
    entity_indices = []
    for start,end,label in annot:
      skip_entity = False
      for idx in range(start,end):
        if idx in entity_indices:
          skip_entity = True
          break
      if skip_entity==True:
        continue
      entity_indices = entity_indices + list(range(start,end))
      try:
        span = doc.char_span(start,end,label=label,alignment_mode='contract')
      except:
        continue
      if span is None:
        err_data = str([start,end]) + ' ' + str(text) + '\n'
        file.write(err_data.encode('UTF-8'))
      else:
        ents.append(span)
    try:
      doc.ents = ents
      db.add(doc)
    except:
      pass
  return db

In [ ]:
from sklearn.model_selection import train_test_split
train,test = train_test_split(cv_load,test_size=0.3)

In [ ]:
len(train) , len(test)

(709, 305)

In [ ]:
file=open('/content/drive/MyDrive/Red_Dot/spacy_CV/train_file.txt','wb')
db = get_spacy_doc(file,train)
db.to_disk('/content/drive/MyDrive/Red_Dot/spacy_CV/train_data.spacy')

db = get_spacy_doc(file,test)
db.to_disk('/content/drive/MyDrive/Red_Dot/spacy_CV/test_data.spacy')

file.close()

100%|██████████| 305/305 [00:02<00:00, 116.97it/s]


In [ ]:
len(db.tokens)

305

In [ ]:
!python -m spacy train /content/drive/MyDrive/Red_Dot/spacy_CV/data/training/config.cfg --output /content/drive/MyDrive/Red_Dot/spacy_CV/finaloutput --paths.train /content/drive/MyDrive/Red_Dot/spacy_CV/train_data.spacy --paths.dev /content/drive/MyDrive/Red_Dot/spacy_CV/test_data.spacy --gpu-id 0

2023-06-19 06:12:28.211384: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Created output directory:
/content/drive/MyDrive/Red_Dot/spacy_CV/finaloutput
ℹ Saving to output directory:
/content/drive/MyDrive/Red_Dot/spacy_CV/finaloutput
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2023-06-19 06:12:36,967] [INFO] Set up nlp object from config
[2023-06-19 06:12:36,984] [INFO] Pipeline: ['transformer', 'ner']
[2023-06-19 06:12:36,987] [INFO] Created vocabulary
[2023-06-19 06:12:36,988] [INFO] Finished initializing nlp object
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializ

In [ ]:
nlp = spacy.load('/content/drive/MyDrive/Red_Dot/spacy_CV/finaloutput/model-best')

Test

In [ ]:
!pip install PyMuPDF
import sys,fitz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 80.2 MB/s eta 0:00:00


In [ ]:
!pip install pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd

In [ ]:
doc = nlp('My name is mumu and i worked in google for 3 years')
for ent in doc.ents:
  print(ent.text,' ->>> ',ent.label_)

In [ ]:
fname = '/content/drive/MyDrive/Red_Dot/spacy_CV/data/test/Alice Clark CV.pdf'
doc=fitz.open(fname)
text = " "
for page in doc:
  text = text + str(page.get_text())
text=text.strip()
text=' '.join(text.split())
print(text)
print('\n')
doc = nlp(text)
for ent in doc.ents:
  print(ent.text,' ->>> ',ent.label_)

Alice Clark AI / Machine Learning Delhi, India Email me on Indeed • 20+ years of experience in data handling, design, and development • Data Warehouse: Data analysis, star/snow flake scema data modelling and design specific to data warehousing and business intelligence • Database: Experience in database designing, scalability, back-up and recovery, writing and optimizing SQL code and Stored Procedures, creating functions, views, triggers and indexes. Cloud platform: Worked on Microsoft Azure cloud services like Document DB, SQL Azure, Stream Analytics, Event hub, Power BI, Web Job, Web App, Power BI, Azure data lake analytics(U-SQL) Willing to relocate anywhere WORK EXPERIENCE Software Engineer Microsoft – Bangalore, Karnataka January 2000 to Present 1. Microsoft Rewards Live dashboards: Description: - Microsoft rewards is loyalty program that rewards Users for browsing and shopping online. Microsoft Rewards members can earn points when searching with Bing, browsing with Microsoft Edge